In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from openpyxl import load_workbook
from sklearn.metrics.pairwise import cosine_similarity
from matcher import BrandMatcher 
from thefuzz import fuzz, process
import re
from rapidfuzz import process, fuzz
import os

C:\Users\tanmay.mishra\AppData\Local\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanmay.mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file_path_automation = "../Output/Exclude Flag Automation.xlsx"

In [3]:
import json

with open('../../../main_config.json') as f:
    main_config = json.load(f)

In [4]:
# === STEP 1: Load the Excel file ===
file_path = "../Output/" + main_config['CATEGORY_NAME'] + "_category_QA_output.xlsx"  # Replace this with the actual path or file upload
output_sheet_name = "Pivot_Output"
xls = pd.ExcelFile(file_path)

In [5]:
df_6sub_dis = pd.read_excel(xls, '6SUB_DIS')
df_agg = pd.read_excel(xls, '9agg_exclude_flag_v1')

# === STEP 2: Choose level (L2 or L3) ===
level = 'L2'  # Change to 'L2' if needed

# Filter relevant entries
relevant_levels = df_6sub_dis[df_6sub_dis['Relevant'].str.upper() == 'YES'][level].dropna().unique()

# === STEP 3: Filter 9agg_exclude_flag_v1 based on level and relevance ===
df_filtered = df_agg[df_agg[level].isin(relevant_levels)]

In [6]:
df_filtered["O_SALE"].sum()

5243670411.801369

In [7]:
df_filtered["O_UNIT"].sum()

2299711473.3599997

In [8]:
df_filtered["TOTAL_SPEND"].sum()

18442859.060400903

In [9]:
# === STEP 4: Create pivot grouped only by UNIQUE_BRAND_NAME ===
pivot = df_filtered.pivot_table(
    index='UNIQUE_BRAND_NAME',
    values=['O_SALE', 'TOTAL_SPEND', 'O_UNIT', 'Exclude_Flag', 'Combine_Flag'],
    aggfunc={
        'O_SALE': 'sum',
        'TOTAL_SPEND': 'sum',
        'O_UNIT': 'sum',
        'Exclude_Flag': 'max',
        'Combine_Flag': 'max'
    },
    fill_value=0
).reset_index()

# === STEP 5: Rename columns ===
pivot.rename(columns={
    'O_SALE': 'Sum of O_SALE',
    'TOTAL_SPEND': 'Sum of TOTAL_SPEND',
    'O_UNIT': 'Sum of O_UNIT',
    'Exclude_Flag': 'Max of Exclude_Flag',
    'Combine_Flag': 'Max of Combine_Flag'
}, inplace=True)

# === STEP 6: Reorder columns ===
data_cols = ['UNIQUE_BRAND_NAME', 'Sum of O_SALE', 'Sum of TOTAL_SPEND',
             'Sum of O_UNIT', 'Max of Exclude_Flag', 'Max of Combine_Flag']
pivot = pivot[data_cols]


In [10]:
pivot.head()

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag
0,0,820.55,123539.658572,34.0,1,0.0
1,4SISTERS,0.00,192.897949,0.0,1,0.0
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0.0
3,AL AMIR,11191.43,0.000000,6577.0,1,0.0
4,AL'FEZ,7113.12,0.000000,2115.0,1,0.0


In [11]:
round(pivot["Sum of O_SALE"].sum(),2)==round(df_filtered["O_SALE"].sum(),2)

True

In [12]:
round(pivot["Sum of TOTAL_SPEND"].sum(),2)==round(df_filtered["TOTAL_SPEND"].sum(),2)

True

In [13]:
round(pivot["Sum of O_UNIT"].sum(),2)==round(df_filtered["O_UNIT"].sum(),2)

True

In [14]:
pivot["Sum of TOTAL_SPEND"].sum()

18442859.060400903

In [15]:
# Calculate total values
total_sale = pivot['Sum of O_SALE'].sum()
total_spend = pivot['Sum of TOTAL_SPEND'].sum()
total_unit = pivot['Sum of O_UNIT'].sum()

# Calculate share percentages and round to 1 decimal place
pivot['O_SALE_share'] = (pivot['Sum of O_SALE'] / total_sale * 100).round(1) if total_sale != 0 else 0
pivot['TOTAL_SPEND_share'] = (pivot['Sum of TOTAL_SPEND'] / total_spend * 100).round(1) if total_spend != 0 else 0
pivot['O_UNIT_share'] = (pivot['Sum of O_UNIT'] / total_unit * 100).round(1) if total_unit != 0 else 0


In [16]:
pivot.head()

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share
0,0,820.55,123539.658572,34.0,1,0.0,0.0,0.7,0.0
1,4SISTERS,0.00,192.897949,0.0,1,0.0,0.0,0.0,0.0
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0.0,0.0,0.0,0.0
3,AL AMIR,11191.43,0.000000,6577.0,1,0.0,0.0,0.0,0.0
4,AL'FEZ,7113.12,0.000000,2115.0,1,0.0,0.0,0.0,0.0


In [17]:
Private_brand = pd.read_excel("Private Brand.xlsx")

In [18]:
Private_brand.head()

,Brand,Label
0,SOLANA,Private Brands
1,MIXED UP,Private Brands
2,VECHERINKA,Private Brands
3,HERMAN SURVIVORS,Private Brands
4,WALMART DELI,Private Brands


In [19]:
mapping_issue_brand= pd.read_excel("Mapping Issue Brand.xlsx")

In [20]:
mapping_issue_brand.head()

,Mapping Issues Brand,Flag
0,0,1
1,UNKNOWN,1
2,UNBRANDED,1
3,ONLINE,1
4,GENERIC,1


In [21]:
pivot.head()

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share
0,0,820.55,123539.658572,34.0,1,0.0,0.0,0.7,0.0
1,4SISTERS,0.00,192.897949,0.0,1,0.0,0.0,0.0,0.0
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0.0,0.0,0.0,0.0
3,AL AMIR,11191.43,0.000000,6577.0,1,0.0,0.0,0.0,0.0
4,AL'FEZ,7113.12,0.000000,2115.0,1,0.0,0.0,0.0,0.0


In [22]:
# Stop words set
stop_words = set(stopwords.words('english'))

# Text cleaner: lowercase, remove non-letters, remove stopwords
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ''.join(words)

# Clean both sides
pivot['brand_cleaned'] = pivot['UNIQUE_BRAND_NAME'].apply(clean_text)
Private_brand['brand_cleaned'] = Private_brand['Brand'].apply(clean_text)

# Fuzzy matching function (fixed unpacking and None check)
def fuzzy_match(brand, choices, threshold=90):  # Decrease threshold if no results
    if brand == "":
        return False
    result = process.extractOne(brand, choices, scorer=fuzz.ratio)
    if result is None:
        return False
    match, score, _ = result  # Unpack all three, ignore index
    return score >= threshold

# Apply fuzzy match and flag
pivot['Pivot Brand Flag'] = pivot['brand_cleaned'].apply(
    lambda x: 1 if fuzzy_match(x, Private_brand['brand_cleaned'].tolist()) else ""
)

# Optional cleanup
pivot.drop(columns=['brand_cleaned'], inplace=True)
Private_brand.drop(columns=['brand_cleaned'], inplace=True)


In [23]:
pivot[pivot["Pivot Brand Flag"]==1]

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share,Pivot Brand Flag
156,FRESHNESS GUARANTEED,4.656458e+04,0.000000,8606.0,1,0.0,0.0,0.0,0.0,1
176,GREAT VALUE,1.321286e+09,11898.187124,652732513.8,1,0.0,25.2,0.1,28.4,1
272,MARKETSIDE,2.016606e+06,0.000000,285686.0,1,0.0,0.0,0.0,0.0,1
390,SAM'S CHOICE,1.969050e+03,0.000000,1211.0,1,0.0,0.0,0.0,0.0,1
497,WAL-MART BAKERY,5.080000e+00,0.000000,2.0,1,0.0,0.0,0.0,0.0,1


In [24]:
import re
from rapidfuzz import process, fuzz

# Assuming stop_words is defined and imported as you showed earlier:
stop_words = set(stopwords.words('english'))

# Text cleaner: lowercase, remove non-letters, remove stopwords
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ''.join(words)

# Clean both sides
pivot['brand_cleaned'] = pivot['UNIQUE_BRAND_NAME'].apply(clean_text)
Private_brand['brand_cleaned'] = Private_brand['Brand'].apply(clean_text)

# === Fuzzy match for Private Brand ===
def fuzzy_match(brand, choices, threshold=90):  # decrease threshold if no result
    if brand == "":
        return False
    result = process.extractOne(brand, choices, scorer=fuzz.ratio)
    if result is None:
        return False
    match, score, _ = result  # unpack three values safely
    return score >= threshold

pivot['Pivot Brand Flag'] = pivot['brand_cleaned'].apply(
    lambda x: 1 if fuzzy_match(x, Private_brand['brand_cleaned'].tolist()) else ""
)

# === Clean Mapping Issues for Exact Match ===
mapping_issue_brand['brand_cleaned'] = mapping_issue_brand['Mapping Issues Brand'].apply(clean_text)
mapping_issues_set = set(mapping_issue_brand['brand_cleaned'].tolist())

pivot['Mapping Issue Flag'] = pivot['brand_cleaned'].apply(
    lambda x: 1 if x in mapping_issues_set else ""
)

# === Final Cleanup ===
pivot.drop(columns=['brand_cleaned'], inplace=True)
# Private_brand is kept intact as per your comment
mapping_issue_brand.drop(columns=['brand_cleaned'], inplace=True)


In [25]:
pivot[pivot["Mapping Issue Flag"]==1]

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share,Pivot Brand Flag,Mapping Issue Flag
0,0,820.55,123539.658572,34.0,1,0.0,0.0,0.7,0.0,,1
89,COMING SOON,15954.61,51.591209,6184.0,0,0.0,0.0,0.0,0.0,,1
162,GENERIC,4315745.85,0.000000,2245683.0,1,0.0,0.1,0.0,0.1,,1
307,NOBRAND,226.17,0.000000,45.0,1,0.0,0.0,0.0,0.0,,1
317,ONLINE,11684.87,0.000000,2474.0,1,0.0,0.0,0.0,0.0,,1
407,SHAN,-13.34,0.000000,-2.0,1,0.0,-0.0,0.0,-0.0,,1
476,UNBRANDED,1292.39,0.000000,497.0,1,0.0,0.0,0.0,0.0,,1


In [26]:
# Import nltk stopwords
import nltk
from nltk.corpus import stopwords

# Download stopwords once
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def normalize_brand(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ''.join(words)

def match_brands_clean(self, new_df, brand_col='UNIQUE_BRAND_NAME'):
    print("Starting clean brand matching...")

    new_embeddings = self.model.encode(new_df[brand_col].tolist(), show_progress_bar=False)
    sim_matrix = cosine_similarity(new_embeddings, self.historical_embeddings)

    if 'Combine_flag' not in new_df.columns:
        new_df['Combine_flag'] = None
    if 'comment' not in new_df.columns:
        new_df['comment'] = None

    for i, (idx, row) in enumerate(new_df.iterrows()):
        brand = row[brand_col]
        best_match_idx = sim_matrix[i].argmax()
        best_score = sim_matrix[i][best_match_idx]
        matched_brand = self.historical_brands[best_match_idx]
        matched_flag = self.historical_map.get(matched_brand, None)
        if best_score >= self.embedding_match_threshold and pd.notna(matched_flag):
            new_df.at[idx, 'Combine_flag'] = matched_flag
            new_df.at[idx, 'comment'] = f'historic match with \"{matched_brand}\" (score: {best_score:.2f})'
    print("Historic brand matching completed.")
    return new_df

# Attach method to BrandMatcher class
BrandMatcher.match_brands_clean = match_brands_clean

# Configuration setup
config = {
    'model_name': 'all-MiniLM-L6-v2',
    'embedding_match_threshold': 0.9,
    'fuzzy_match_threshold': 90,
}
matcher = BrandMatcher(config)
matcher.load_historical_data("combined_output.json")

# Clean brand names for fuzzy matching
pivot['brand_cleaned'] = pivot['UNIQUE_BRAND_NAME'].apply(normalize_brand)

# Filter out rows flagged as 'Pivot Brand Flag' == 1 or 'Mapping Issue Flag' == 1
valid_mask = pd.Series(True, index=pivot.index)
if 'Pivot Brand Flag' in pivot.columns:
    valid_mask &= (pivot['Pivot Brand Flag'] != 1)
if 'Mapping Issue Flag' in pivot.columns:
    valid_mask &= (pivot['Mapping Issue Flag'] != 1)
df_valid = pivot[valid_mask].copy()
df_excluded = pivot[~valid_mask].copy()

# Step 1: Historic embedding-based brand matching
df_valid = matcher.match_brands_clean(df_valid, brand_col='UNIQUE_BRAND_NAME')

# Step 2: Fuzzy matching 
known_brands_flags = {k: v for k, v in matcher.historical_map.items() if pd.notna(v)}
known_cleaned_brands = list(set(normalize_brand(b) for b in known_brands_flags.keys()))
clean_to_orig = {normalize_brand(b): b for b in known_brands_flags.keys() if normalize_brand(b)}

for idx, row in df_valid[df_valid['Combine_flag'].isna()].iterrows():
    brand = row['UNIQUE_BRAND_NAME']
    brand_clean = row['brand_cleaned']
    if not isinstance(brand, str) or not brand.strip() or not brand_clean:
        df_valid.at[idx, 'Combine_flag'] = None
        df_valid.at[idx, 'comment'] = None
        continue
    fuzzy_match = process.extractOne(brand_clean, known_cleaned_brands, scorer=fuzz.ratio)
    if fuzzy_match and fuzzy_match[1] >= config['fuzzy_match_threshold']:
        matched_cleaned = fuzzy_match[0]
        matched_orig = clean_to_orig.get(matched_cleaned, None)
        matched_flag = known_brands_flags.get(matched_orig, None)
        if pd.notna(matched_flag):
            df_valid.at[idx, 'Combine_flag'] = matched_flag
            df_valid.at[idx, 'comment'] = f'fuzzy match with \"{matched_orig}\" (score: {fuzzy_match[1]:.2f})'
        else:
            df_valid.at[idx, 'Combine_flag'] = None
            df_valid.at[idx, 'comment'] = None
    else:
        df_valid.at[idx, 'Combine_flag'] = None
        df_valid.at[idx, 'comment'] = None

print("Fuzzy brand matching completed.")

# Only keep Combine_flag groups with multiple members, and remove comments for singletons
df_valid['Combine_flag'] = df_valid['Combine_flag'].astype('Int64')
flag_counts = df_valid['Combine_flag'].value_counts()
flags_to_keep = flag_counts[flag_counts > 1].index.tolist()
singleton_mask = ~df_valid['Combine_flag'].isin(flags_to_keep)
df_valid.loc[singleton_mask, 'Combine_flag'] = pd.NA
df_valid.loc[singleton_mask, 'comment'] = None

# Normalize Combine_flag to continuous integers starting at 1
unique_flags = sorted(df_valid['Combine_flag'].dropna().unique())
combine_map_seq = {old: new for new, old in enumerate(unique_flags, start=1)}
df_valid['Combine_flag'] = df_valid['Combine_flag'].map(combine_map_seq)

# Assign numeric Combine Flag column, keep NaN as blank for ungrouped
df_valid['Combine Flag'] = df_valid['Combine_flag'].astype('Int64')

# For excluded rows, assign blank Combine Flag and comment
df_excluded['Combine Flag'] = ''
if 'comment' not in df_excluded.columns:
    df_excluded['comment'] = ''
else:
    df_excluded['comment'] = df_excluded['comment'].fillna('')

# Drop helper columns before merging back
for col in ['Combine_flag', 'brand_cleaned']:
    if col in df_valid.columns:
        df_valid.drop(columns=[col], inplace=True)
    if col in df_excluded.columns:
        df_excluded.drop(columns=[col], inplace=True)

# Concatenate valid and excluded back into full pivot
pivot = pd.concat([df_valid, df_excluded], axis=0).sort_index()

# Prepare brand analysis sheet with requested columns
brand_analysis_cols = [
    'UNIQUE_BRAND_NAME',
    'Sum of O_SALE',
    'Sum of TOTAL_SPEND',
    'Sum of O_UNIT',
    'Max of Exclude_Flag',
    'Combine Flag',
    'comment'
]
available_cols = [c for c in brand_analysis_cols if c in pivot.columns]
brand_analysis = pivot[available_cols].drop_duplicates(subset=['UNIQUE_BRAND_NAME']).reset_index(drop=True)

# Save brand_analysis to Excel
output_path = "../Output/Exclude Flag Automation.xlsx"
from openpyxl import load_workbook
if os.path.exists(output_path):
    with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        brand_analysis.to_excel(writer, sheet_name="brand_analysis", index=False)
else:
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        brand_analysis.to_excel(writer, sheet_name="brand_analysis", index=False)

print(f"Brand analysis saved to {output_path}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanmay.mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading cache from parent_cache.pkl...
No cache file found at parent_cache.pkl, starting with empty cache.
Loading historical data from combined_output.json...
Encoding embeddings for historical brands...


Batches:   0%|          | 0/127 [00:00<?, ?it/s]

Historical data loaded. Next Combine flag: 1595
Starting clean brand matching...
Historic brand matching completed.
Fuzzy brand matching completed.
Brand analysis saved to ../Output/Exclude Flag Automation.xlsx


In [27]:
pivot[pivot["Combine Flag"]==1]

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share,Pivot Brand Flag,Mapping Issue Flag,comment,Combine Flag
173,GRACE,109387.19,327.599999,78623.0,0,0.0,0.0,0.0,0.0,,,"historic match with ""grace"" (score: 1.00)",1
174,GRACE FOODS,74870.95,0.760000,26479.0,0,0.0,0.0,0.0,0.0,,,"historic match with ""grace foods"" (score: 1.00)",1


In [28]:
# Initialize as blank
pivot['update_exclude_flag'] = ""

# 1. Mapping Issue Flag
pivot.loc[pivot['Mapping Issue Flag'] == 1, 'update_exclude_flag'] = 1

# 2. Pivot Brand Flag
pivot.loc[pivot['Pivot Brand Flag'] == 1, 'update_exclude_flag'] = 1

# 3. Combine Flag is NOT null → force 0 if not already 1
pivot.loc[
    (pivot['Combine Flag'].notnull()) & (pivot['update_exclude_flag'] == ""),
    'update_exclude_flag'
] = 0

# 4. Combine Flag IS null and (Spend == 0 or Sale == 0 or Unit == 0) → set 1
pivot.loc[
    (pivot['Combine Flag'].isnull()) &
    (
        (pivot['Sum of TOTAL_SPEND'] == 0) |
        (pivot['Sum of O_SALE'] == 0) |
        (pivot['Sum of O_UNIT'] == 0)
    ) &
    (pivot['update_exclude_flag'] == ""),
    'update_exclude_flag'
] = 1

# 5. Remaining rows → set 0
pivot.loc[pivot['update_exclude_flag'] == "", 'update_exclude_flag'] = 0

# Convert to int
pivot['update_exclude_flag'] = pivot['update_exclude_flag'].astype(int)


In [29]:
pivot

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,O_SALE_share,TOTAL_SPEND_share,O_UNIT_share,Pivot Brand Flag,Mapping Issue Flag,comment,Combine Flag,update_exclude_flag
0,0,820.55,123539.658572,34.0,1,0.0,0.0,0.7,0.0,,1,,,1
1,4SISTERS,0.00,192.897949,0.0,1,0.0,0.0,0.0,0.0,,,None,<NA>,1
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0.0,0.0,0.0,0.0,,,None,<NA>,0
3,AL AMIR,11191.43,0.000000,6577.0,1,0.0,0.0,0.0,0.0,,,None,<NA>,1
4,AL'FEZ,7113.12,0.000000,2115.0,1,0.0,0.0,0.0,0.0,,,None,<NA>,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,YEHUDA,5578.92,0.000000,582.0,1,0.0,0.0,0.0,0.0,,,None,<NA>,1
508,YELLOWSTONE,9001621.47,45373.781226,2845864.0,0,0.0,0.2,0.2,0.1,,,None,<NA>,0
509,YODER'S,53555.96,161.859320,31988.0,0,0.0,0.0,0.0,0.0,,,None,<NA>,0
510,YUMMY,131.49,0.000000,27.0,1,0.0,0.0,0.0,0.0,,,None,<NA>,1


In [30]:
# Step 1: Rename columns
pivot.rename(columns={
    'O_SALE_share': 'Sales Share',
    'O_UNIT_share': 'Unit Share',
    'TOTAL_SPEND_share': 'Spend Share',
    'Pivot Brand Flag': 'Private Brand',
    'Mapping Issue Flag': 'Mapping Issue',
    'update_exclude_flag': 'Exclude Flag'
}, inplace=True)

# Step 2: Reorder columns
new_column_order = [
    'UNIQUE_BRAND_NAME',
    'Sum of O_SALE',
    'Sum of TOTAL_SPEND',
    'Sum of O_UNIT',
    'Max of Exclude_Flag',
    'Max of Combine_Flag',
    'Sales Share',
    'Unit Share',
    'Spend Share',
    'Private Brand',
    'Mapping Issue',
    'Combine Flag',
    'Exclude Flag'
]

# Apply the new order
pivot = pivot[new_column_order]

In [31]:
pivot.head()

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,Sales Share,Unit Share,Spend Share,Private Brand,Mapping Issue,Combine Flag,Exclude Flag
0,0,820.55,123539.658572,34.0,1,0.0,0.0,0.0,0.7,,1,,1
1,4SISTERS,0.00,192.897949,0.0,1,0.0,0.0,0.0,0.0,,,<NA>,1
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0.0,0.0,0.0,0.0,,,<NA>,0
3,AL AMIR,11191.43,0.000000,6577.0,1,0.0,0.0,0.0,0.0,,,<NA>,1
4,AL'FEZ,7113.12,0.000000,2115.0,1,0.0,0.0,0.0,0.0,,,<NA>,1


In [32]:
output_path = "../Output/exclude_flag_analysis.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    pivot.to_excel(writer, sheet_name="analysis", index=False)

print(f"Excel file saved at: {output_path}")

Excel file saved at: ../Output/exclude_flag_analysis.xlsx


In [33]:
# === Step 1: Read source data from Pivot_Output sheet ===
source_file_path = "../Output/exclude_flag_analysis.xlsx"#change
source_sheet_name = "analysis"

# Read data
df_pivot = pd.read_excel(source_file_path, sheet_name=source_sheet_name)

# === Step 2: Rename and reorder columns ===
df_pivot.rename(columns={
    'O_SALE_share': 'Sales Share',
    'O_UNIT_share': 'Unit Share',
    'TOTAL_SPEND_share': 'Spend Share',
    'Pivot Brand Flag': 'Private Brand',
    'Mapping Issue Flag': 'Mapping Issue',
    'update_exclude_flag': 'Exclude Flag'
}, inplace=True)

# Define final column order
final_columns = [
    'UNIQUE_BRAND_NAME',
    'Sum of O_SALE',
    'Sum of TOTAL_SPEND',
    'Sum of O_UNIT',
    'Max of Exclude_Flag',
    'Max of Combine_Flag',
    'Sales Share',
    'Unit Share',
    'Spend Share',
    'Private Brand',
    'Mapping Issue',
    'Combine Flag',
    'Exclude Flag'
]

# Keep only those columns that exist in source
updated_data = df_pivot[[col for col in final_columns if col in df_pivot.columns]]

# === Step 3: Load target Excel file and sheet ===
 
target_sheet_name = "Analysis"

wb = load_workbook(file_path_automation)
ws = wb[target_sheet_name]

# === Step 4: Map headers to their column indexes ===
headers = [cell.value for cell in ws[1]]
header_to_col = {header: idx + 1 for idx, header in enumerate(headers)}

# Define which columns to update based on the updated_data
data_cols = list(updated_data.columns)

# === Step 5: Clear ONLY the data area (rows 2 down) for selected columns ===
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    for col_name in data_cols:
        col_idx = header_to_col.get(col_name)
        if col_idx:
            row[col_idx - 1].value = None  # Clear cell value, keep formulas intact

# === Step 6: Write the updated data (starting from row 2) ===
start_row = 2
for i, row_data in updated_data.iterrows():
    for col_name in data_cols:
        col_idx = header_to_col.get(col_name)
        if col_idx:
            ws.cell(row=start_row + i, column=col_idx).value = row_data[col_name]

# === Step 7: Save the workbook ===
wb.save(file_path_automation)
print("✅ Data replaced into 'Analysis' sheet (formulas untouched).")

✅ Data replaced into 'Analysis' sheet (formulas untouched).


In [34]:
file_path_automation

'../Output/Exclude Flag Automation.xlsx'

In [35]:
sheet_name = "Analysis"

# Step 1: Read and clean
pivot = pd.read_excel(file_path_automation, sheet_name=sheet_name)
pivot = pivot.loc[:, ~pivot.columns.str.startswith('Unnamed')]

# Step 2: Filter & sort Private Brands
top_private_brands = pivot[pivot["Private Brand"] == 1]
top_private_brands.sort_values(by="Sum of O_SALE", ascending=False, inplace=True)


# Step 3: Remove old sheet if it exists
wb = load_workbook(file_path_automation)
if "Top Private Brands" in wb.sheetnames:
    del wb["Top Private Brands"]
    wb.save(file_path_automation)

# Step 4: Write new sheet
with pd.ExcelWriter(file_path_automation, engine='openpyxl', mode='a') as writer:
    top_private_brands.to_excel(writer, sheet_name="Top Private Brands", index=False)

print("✅ 'Top Private Brands' sheet created cleanly without Unnamed columns.")


C:\Users\tanmay.mishra\AppData\Local\Temp\ipykernel_7636\2793394549.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_private_brands.sort_values(by="Sum of O_SALE", ascending=False, inplace=True)


✅ 'Top Private Brands' sheet created cleanly without Unnamed columns.


In [36]:

sheet_name = "Analysis"

# === Step 1: Read and clean data ===
pivot = pd.read_excel(file_path_automation, sheet_name=sheet_name)
pivot = pivot.loc[:, ~pivot.columns.str.startswith('Unnamed')]  # Remove unwanted columns

# === Step 2: Filter Mapping Issue Brands ===
mapping_issue_brands = pivot[pivot["Mapping Issue"] == 1]
mapping_issue_brands.sort_values(by="Sum of O_SALE", ascending=False, inplace=True)
# === Step 3: Remove old sheet if exists ===
wb = load_workbook(file_path_automation)
if "Mapping Issue Brands" in wb.sheetnames:
    del wb["Mapping Issue Brands"]
    wb.save(file_path_automation)

# === Step 4: Write to new sheet ===
with pd.ExcelWriter(file_path_automation, engine='openpyxl', mode='a') as writer:
    mapping_issue_brands.to_excel(writer, sheet_name="Mapping Issue Brands", index=False)

print("✅ 'Mapping Issue Brands' sheet created successfully.")


C:\Users\tanmay.mishra\AppData\Local\Temp\ipykernel_7636\1346993221.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_issue_brands.sort_values(by="Sum of O_SALE", ascending=False, inplace=True)


✅ 'Mapping Issue Brands' sheet created successfully.


In [37]:
code_output = pd.read_excel("../Output/exclude_flag_analysis.xlsx")

In [38]:
code_output

,UNIQUE_BRAND_NAME,Sum of O_SALE,Sum of TOTAL_SPEND,Sum of O_UNIT,Max of Exclude_Flag,Max of Combine_Flag,Sales Share,Unit Share,Spend Share,Private Brand,Mapping Issue,Combine Flag,Exclude Flag
0,0,820.55,123539.658572,34.0,1,0,0.0,0.0,0.7,NaN,1.0,NaN,1
1,4SISTERS,0.00,192.897949,0.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN,1
2,A DOZEN COUSINS,5014.99,11.320000,227.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN,0
3,AL AMIR,11191.43,0.000000,6577.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN,1
4,AL'FEZ,7113.12,0.000000,2115.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,YEHUDA,5578.92,0.000000,582.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN,1
508,YELLOWSTONE,9001621.47,45373.781226,2845864.0,0,0,0.2,0.1,0.2,NaN,NaN,NaN,0
509,YODER'S,53555.96,161.859320,31988.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN,0
510,YUMMY,131.49,0.000000,27.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN,1


In [39]:
sheet_name = "code_output"

# === Step 1: Clean Unnamed Columns (optional if already done) ===
code_output = code_output.loc[:, ~code_output.columns.str.startswith('Unnamed')]

# === Step 2: Delete sheet if already exists ===
wb = load_workbook(file_path_automation)
if sheet_name in wb.sheetnames:
    del wb[sheet_name]
    wb.save(file_path_automation)

# === Step 3: Write to sheet ===
with pd.ExcelWriter(file_path_automation, engine='openpyxl', mode='a') as writer:
    code_output.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ 'code_output' written to Excel successfully.")


✅ 'code_output' written to Excel successfully.


In [40]:

# Read the pivot data (assuming you've already loaded it earlier as `pivot`)
# Filter Combine Brands (where Combine Flag is NOT null)
combine_brand = pivot[pivot["Combine Flag"].notna()]

# Sort by Combine Flag (optional)
combine_brand = combine_brand.sort_values(by="Combine Flag", ascending=True)

# Write to Excel in a new sheet
with pd.ExcelWriter(file_path_automation, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    combine_brand.to_excel(writer, sheet_name="Combine Brands", index=False)

print("✅ 'Combine Brands' sheet successfully added to Exclude Flag Automation.xlsx.")


✅ 'Combine Brands' sheet successfully added to Exclude Flag Automation.xlsx.
